In [85]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['indiahackathon_test_20180928.txt', 'indiahackathon_train_20180928.txt']


In [86]:
from fastai.text import *

In [87]:
import html

In [88]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

In [89]:
train_file_path = "../input/indiahackathon_train_20180928.txt"

In [90]:
test_file_path = "../input/indiahackathon_test_20180928.txt"

In [91]:
#Library Imports
import numpy as np 
import pandas as pd 
import bz2
import gc
import chardet
import re
import os

In [92]:
#Keras Imports
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input, Conv1D, GlobalMaxPool1D, Dropout, concatenate, Layer, InputSpec, CuDNNLSTM
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras import activations, initializers, regularizers, constraints
from keras.utils.conv_utils import conv_output_length
from keras.regularizers import l2
from keras.constraints import maxnorm

In [93]:
#Read Train File
with open(train_file_path) as train_file:
    train_file_data = train_file.readlines()
    
train_file_data[0]

'__label__2 The Conspirator: It is a shame this movie was not given the praise it deserves. Robert Redford did an excellent job, while the actors in this movie wer not household names they all gave a wonderful and believable performance. I would suggest all history buffs view this masterpiece. We took my then 12 year old granddaughter to the theatre to see this and she just loved it. Wonderful movie well done.\n'

In [94]:
#Read Test File
with open(test_file_path) as test_file:
    test_file_data = test_file.readlines()
    
test_file_data[0]

"STAY AWAY: I lost over 300 excellent shots thanks to this card. I've tried everything -- the pictures come back corrupted every time. Sometimes it will be just a few pictures, other times it's the whole card.PNY Tech Support is absolutlely the worst I've ever experienced in my life. 2+ hour waits on hold to get hung up on. What good is a lifetime warranty if you can't get through to the company!Stay far away from all PNY products -- that's what I'm doing forever.\n"

In [95]:
#Prepare Training Data

train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_data]
train_sentences = [x.lower() for x in train_file_data]

In [96]:
train_labels[:5]

[1, 0, 0, 1, 1]

In [97]:
train_sentences[:5]

['__label__2 the conspirator: it is a shame this movie was not given the praise it deserves. robert redford did an excellent job, while the actors in this movie wer not household names they all gave a wonderful and believable performance. i would suggest all history buffs view this masterpiece. we took my then 12 year old granddaughter to the theatre to see this and she just loved it. wonderful movie well done.\n',
 "__label__1 don't waste your money!: this router lasted about a year then it started dropping wireless signals. it also had to be rebooted all the time. it was very aggravating having to reboot the router all the time and had to be in the same room for the wireless to work. i replaced it with a cisco-linksys wrt54gl router that is working without a hitch!\n",
 "__label__1 wow...: i honestly only bought this for comedy purposes, but i had no idea what i was in for!this is possibly one of the worst cds ever produced, tribute or otherwise. the orchestration is incredibly sub p

In [98]:
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

In [99]:
train_sentences[:5]

['__label__0 the conspirator: it is a shame this movie was not given the praise it deserves. robert redford did an excellent job, while the actors in this movie wer not household names they all gave a wonderful and believable performance. i would suggest all history buffs view this masterpiece. we took my then 00 year old granddaughter to the theatre to see this and she just loved it. wonderful movie well done.\n',
 "__label__0 don't waste your money!: this router lasted about a year then it started dropping wireless signals. it also had to be rebooted all the time. it was very aggravating having to reboot the router all the time and had to be in the same room for the wireless to work. i replaced it with a cisco-linksys wrt00gl router that is working without a hitch!\n",
 "__label__0 wow...: i honestly only bought this for comedy purposes, but i had no idea what i was in for!this is possibly one of the worst cds ever produced, tribute or otherwise. the orchestration is incredibly sub p

In [100]:
with open(test_file_path) as test_file:
    test_file_data = test_file.readlines()
#Same for Test Data
#test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_data]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file_data]

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

In [101]:
test_sentences[0]

"away: i lost over 000 excellent shots thanks to this card. i've tried everything -- the pictures come back corrupted every time. sometimes it will be just a few pictures, other times it's the whole card.pny tech support is absolutlely the worst i've ever experienced in my life. 0+ hour waits on hold to get hung up on. what good is a lifetime warranty if you can't get through to the company!stay far away from all pny products -- that's what i'm doing forever."

In [102]:
#Some more preprocessing
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [103]:
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [104]:
#Free up memory
del train_file_data, test_file_data

In [105]:
gc.collect()

279

In [106]:
#Get started with hyper params
max_features = 20000
maxlen = 100

In [107]:
tokenizer = text.Tokenizer(num_words=max_features)

In [108]:
tokenizer.fit_on_texts(train_sentences)

In [109]:
tokenized_train = tokenizer.texts_to_sequences(train_sentences)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [110]:
X_train[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    11,
          10,     1, 12828,     6,     9,     4,   800,     8,    63,    15,    16,   497,     1,  2644,
           6,  1538,  1505, 17894,   100,    43,   198,   340,   155,     1,  1004,    12,     8,    63,
       12829,    16,  4527,  1539,    35,    29,   383,     4,   257,     2,  1809,   625,     3,    41,
         835,    29,   348,  5612,   758,     8,  1540,    72,   327,    22,   107,    46,   180,   134,
        3206,     5,     1,  3474,     5,   125,     8,     2,    89,    37,   235,     6,   257,    63,
          70,   307], dtype=int32)

In [111]:
tokenized_test = tokenizer.texts_to_sequences(test_sentences)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [112]:
del tokenized_test, tokenized_train, tokenizer, train_sentences, test_sentences
gc.collect()

0

In [113]:
X_test[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   229,     3,   420,   111,   169,   198,  2661,   711,     5,
           8,   591,   141,   262,   264,     1,   432,   243,   114, 10093,   152,    55,   582,     6,
          50,    27,    37,     4,   164,   432,    77,   206,    51,     1,   322,   591,  1244,   563,
           9,     1,   300,   141,   135,  1593,    12,    22,   153,    10,   764, 14804,    20,   419,
           5,    52,  5293,    53,    20,    48,    34,     9,     4,  2061,  1163,    30,    17,   163,
          52,   151,     5,     1,   512,   543,   228,   229,    36,    29,   622,   315,    48,   113,
         436,  1080], dtype=int32)

In [114]:
batch_size = 32
epochs = 10

In [31]:
def cudnnlstm_model(conv_layers = 2, max_dilation_rate = 3):
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Dropout(0.25)(x)
    x = Conv1D(2*embed_size, kernel_size = 3)(x)
    prefilt = Conv1D(2*embed_size, kernel_size = 3)(x)
    x = prefilt
    for strides in [1, 1, 2]:
        x = Conv1D(128*2**(strides), strides = strides, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_size=3, kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x_f = CuDNNLSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)  
    x_b = CuDNNLSTM(512, kernel_regularizer=l2(4e-6), bias_regularizer=l2(4e-6), kernel_constraint=maxnorm(10), bias_constraint=maxnorm(10))(x)
    x = concatenate([x_f, x_b])
    x = Dropout(0.5)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['binary_accuracy'])

    return model

SyntaxError: invalid syntax (<ipython-input-31-ab236ff36331>, line 16)

In [ ]:
cudnnlstm_model = cudnnlstm_model()
cudnnlstm_model.summary()

In [ ]:
weight_path="early_weights.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks = [checkpoint, early_stopping]

In [ ]:
cudnnlstm_model.fit(X_train, train_labels, batch_size=batch_size, epochs=epochs, shuffle = True, validation_split=0.15, callbacks=callbacks)

In [ ]:
cudnnlstm_model.load_weights(weight_path)
score, acc = cudnnlstm_model.evaluate(X_test, test_labels, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
result = cudnnlstm_model.predict(X_test)

In [ ]:
len(result)

In [ ]:
test_sentences[11000]

In [147]:
result_binarylog = []
for output in predicted:
    if output < 0.5:
        result_binarylog.append("__label__1")
    else:
        result_binarylog.append("__label__2")

In [ ]:
result[20000-1]

In [ ]:
result_binary[20000-34:]

In [148]:
with open("result2.txt", "w") as file:
    file.write(result_binary[0])
    for results in result_binary[1:]:
        file.write('\n')
        file.write(results)

In [149]:
!cat result2.txt

__label__1
__label__2
__label__1
__label__2
__label__1
__label__2
__label__2
__label__2
__label__2
__label__1
__label__1
__label__2
__label__1
__label__2
__label__2
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2
__label__2
__label__1
__label__2
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2
__label__2
__label__1
__label__1
__label__2
__label__1
__label__2
__label__1
__label__2
__label__2
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__2
__label__2
__label__1
__label__1
__label__1
__label__2
__label__2
__label__1
__label__2
__label__2
__label__2
__label__2
__label__1
__label__2
__label__2
__label__2
__label__2
__label__2
__label__2
__label__2
__label__2
__label__2
__label__1
__label__1
__label__2
__label__2
__label__2
__label__2
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2
__label__1

Naive bayes

In [142]:
from sklearn.naive_bayes import GaussianNB

In [143]:
classifier1=GaussianNB()

In [144]:
classifier1.fit(X_train,train_labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [145]:
predicted=classifier1.predict(X_test)

In [146]:
predicted

array([0, 0, 0, ..., 0, 0, 1])

In [64]:
with open("resultbayes.txt", "w") as file:
    file.write(result_binarybayes[0])
    for results in result_binarybayes[1:]:
        file.write('\n')
        file.write(results)

Logistic Regression

In [66]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=0.25, dual=True)
model.fit(X_train, train_labels)
preds = model.predict(X_test)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [67]:
preds

array([1, 0, 0, ..., 0, 1, 0])

Catboost

In [126]:
from catboost import CatBoostClassifier

In [157]:
model=CatBoostRegressor(iterations=50, depth=4, learning_rate=0.1, loss_function='RMSE')

NameError: name 'CatBoostRegressor' is not defined

In [158]:
model.fit(X_train, train_labels)

0:	learn: 0.6824289	total: 419ms	remaining: 20.5s
1:	learn: 0.6719410	total: 748ms	remaining: 17.9s
2:	learn: 0.6623100	total: 1.12s	remaining: 17.6s
3:	learn: 0.6589844	total: 1.45s	remaining: 16.6s
4:	learn: 0.6514525	total: 1.78s	remaining: 16s
5:	learn: 0.6460376	total: 2.1s	remaining: 15.4s
6:	learn: 0.6429508	total: 2.63s	remaining: 16.2s
7:	learn: 0.6396343	total: 3.19s	remaining: 16.8s
8:	learn: 0.6359703	total: 3.78s	remaining: 17.2s
9:	learn: 0.6253919	total: 4.38s	remaining: 17.5s
10:	learn: 0.6211871	total: 4.97s	remaining: 17.6s
11:	learn: 0.6186239	total: 5.55s	remaining: 17.6s
12:	learn: 0.6141059	total: 6.12s	remaining: 17.4s
13:	learn: 0.6102600	total: 6.69s	remaining: 17.2s
14:	learn: 0.6068554	total: 7.25s	remaining: 16.9s
15:	learn: 0.6006932	total: 7.82s	remaining: 16.6s
16:	learn: 0.5949902	total: 8.4s	remaining: 16.3s
17:	learn: 0.5927995	total: 8.98s	remaining: 16s
18:	learn: 0.5887267	total: 9.55s	remaining: 15.6s
19:	learn: 0.5856935	total: 10.1s	remaining: 15

In [159]:
catpredict=model.predict(X_test)

In [160]:
catpredict

array([0., 1., 0., ..., 1., 0., 0.])

In [161]:
result_binary = []
for output in catpredict:
    if output < 0.5:
        result_binary.append("__label__1")
    else:
        result_binary.append("__label__2")

In [162]:
with open("resultcat.txt", "w") as file:
    file.write(result_binary[0])
    for results in result_binary[1:]:
        file.write('\n')
        file.write(results)

In [163]:
!cat resultbayes.txt

__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__2
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__2
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1

XGBClassifier

In [115]:
from xgboost import XGBClassifier

In [116]:
model2=XGBClassifier()

In [117]:
model2.fit(X_train,train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [118]:
xgpred=model.predict(X_test)

In [119]:
xgpred

array([1, 0, 0, ..., 0, 1, 0])

In [123]:
result_binary = []
for output in xgpred:
    if output < 0.5:
        result_binary.append("__label__1")
    else:
        result_binary.append("__label__2")

In [124]:
with open("resultxg.txt", "w") as file:
    file.write(result_binary[0])
    for results in result_binary[1:]:
        file.write('\n')
        file.write(results)

In [125]:
!cat resultxg.txt

__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__2
__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__2
__label__2
__label__1
__label__1
__label__2
__label__2
__label__2
__label__1
__label__2
__label__2
__label__1
__label__1
__label__2
__label__1
__label__2
__label__2
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__1
__label__2
__label__1
__label__2
__label__1
__label__1
__label__1
__label__2
__label__1
__label__2
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__2
__label__1
__label__2
__label__2
__label__1
__label__2
__label__1
__label__2
__label__1
__label__1
__label__2
__label__1
__label__2
__label__2
__label__1
__label__2
__label__1
__label__2
__label__1
__label__1
__label__2
__label__1
__label__1
__label__1
__label__1
__label__1
__label__1
__label__2
__label__2